Run this after untar

In [ ]:
# # Find directories in reverse order of depth, rename directories
# find LR_Landcover HR_Landcover -type d | tac | while IFS= read -r dir
# do
#     new_dir="${dir// /_}"
#     if [ "$dir" != "$new_dir" ]; then
#         mv "$dir" "$new_dir"
#     fi
# done

# # Find files and rename files
# find LR_Landcover HR_Landcover -type f | while IFS= read -r file
# do
#     new_file="${file// /_}"
#     if [ "$file" != "$new_file" ]; then
#         mv "$file" "$new_file"
#     fi
# done


In [ ]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
import pandas as pd
import rasterio
import numpy as np

out_dir = os.path.join(os.path.expanduser("~"), "segment-anything-services/model-weights")
checkpoint = os.path.join(out_dir, "sam_vit_h_4b8939.pth")

sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    sam_kwargs=None,
)

worldstrat_df = pd.read_csv("../data/metadata.csv")

def sample_rows(df):
    sampled_df = df.groupby('IPCC Class').apply(lambda x: x.sample(n=5))
    sampled_df.reset_index(drop=True, inplace=True)
    return sampled_df


def add_image_paths(df, band_order=[3,2,1], revisit_number=1):
    # Define the path template
    path_template = f"../data/HR_Landcover/{{scene_id}}/{{scene_id}}_rgb.png"

    # Add HR_path column
    df['HR_path'] = df['ID'].apply(lambda x: path_template.format(scene_id=x))
    
    path_template = f"../data/LR_Landcover/{{scene_id}}/L2A/"
    # Add LR_path column. this contains multiple revisits so it's to the directory instead of a file
    df['LR_folder_path'] = df['ID'].apply(lambda x: path_template.format(scene_id=x))
    
    # Add LR_path column. this contains multiple revisits so it's to the directory instead of a file
    path_template = f"../data/LR_Landcover/{{scene_id}}/L2A/{{scene_id}}-{{revisit_number}}-L2A_data.tiff"
    df['LR_revisit_1'] = df['ID'].apply(lambda x: path_template.format(scene_id=x, revisit_number=revisit_number))

    return df

In [ ]:
df = sample_rows(worldstrat_df)
df = df.rename(columns={"Unnamed: 0": "ID"})
df = add_image_paths(df)
df['ID'] = df['ID'].str.replace(' ', '_')
df['HR_path'] = df['HR_path'].str.replace(' ', '_')
df['LR_folder_path'] = df['LR_folder_path'].str.replace(' ', '_')
df['LR_revisit_1'] = df['LR_revisit_1'].str.replace(' ', '_')

In [ ]:
def convert_tif_dtype(file_path, band_order = [3,2,1]):
    # Open the image file
    with rasterio.open(file_path) as src:
        image = src.read()  # 3D array: (bands, height, width)
        meta = src.meta

    # If the image data type is float32, convert it to uint8
    if image.dtype == np.float32:
        image = image[band_order,:,:] # rgb image
        meta['count'] = 3
        # Scale float32 array to 0-255 and convert to uint8
        image = ((image - image.min()) / (image.max() - image.min()) * 255).astype(np.uint8)

        # Update the metadata
        meta.update(dtype=rasterio.uint8)

        # Construct the output file path
        file_dir = os.path.dirname(file_path)
        file_base = os.path.basename(file_path)
        file_name, file_ext = os.path.splitext(file_base)
        out_path = os.path.join(file_dir, f"{file_name}_uint8_bo{'-'.join(map(str, band_order))}{file_ext}")

        # Write the updated image to a new file
        with rasterio.open(out_path, 'w', **meta) as dst:
            dst.write(image)

    return out_path

In [ ]:
def convert_dtype_and_add_to_df(df):
    # Initialize an empty list to store the output file paths
    output_file_paths = []

    # Iterate over the DataFrame
    for idx, row in df.iterrows():
        # Construct the full file path
        file_path = row['LR_revisit_1']

        # Check if the file is a tiff file
        if file_path.endswith('.tiff') and "L2A_data" in file_path:
            # Convert the tiff file if needed and get the output file path
            output_file_path = convert_tif_dtype(file_path)

            # Append the output file path to the list
            output_file_paths.append(output_file_path)
        else:
            # If the file is not a tiff file or does not contain "L2A_data", append a None
            output_file_paths.append(None)

    # Add the output file paths as a new column in the DataFrame
    df['LR_output_path'] = output_file_paths

    return df

df = convert_dtype_and_add_to_df(df)

In [ ]:
def generate_masks_zero_shot(df, sam, out_dir, band_order=[3,2,1], revisit_num=1):
    # Create output directories for HR and LR masks
    hr_out_dir = os.path.join(out_dir, 'HR')
    lr_out_dir = os.path.join(out_dir, 'LR')

    os.makedirs(hr_out_dir, exist_ok=True)
    os.makedirs(lr_out_dir, exist_ok=True)

    # Add new columns for the mask paths
    df['HR_mask_path'] = None
    df['LR_mask_path'] = None

    for idx, row in df.iterrows():
        # Get image paths
        hr_image_path = row['HR_path']
        lr_image_path = row['LR_output_path']
        
        # Generate unique IDs for masks, based on image ID
        hr_image_id = os.path.splitext(os.path.basename(hr_image_path))[0]
        lr_image_id = os.path.splitext(os.path.basename(lr_image_path))[0]
        hr_mask_path = os.path.join(hr_out_dir, f"{hr_image_id}_mask.tif")
        lr_mask_path = os.path.join(lr_out_dir, f"{lr_image_id}_mask.tif")
        
        # Generate masks
        sam.generate(hr_image_path, hr_mask_path, batch=True, foreground=True, 
                     erosion_kernel=(3, 3), mask_multiplier=255, unique=True)
        sam.generate(lr_image_path, lr_mask_path, batch=True, foreground=True, 
                     erosion_kernel=(3, 3), mask_multiplier=255, unique=True)
        
        # Save mask paths to the DataFrame
        df.loc[idx, 'HR_mask_path'] = hr_mask_path
        df.loc[idx, 'LR_mask_path'] = lr_mask_path
        df.loc[idx, 'LR_image_path'] = lr_image_path

    # Save the updated DataFrame
    df.to_csv(f'{out_dir}/sample_dataframe_with_zeroshot_masks.csv', index=False)
    return df

generate_masks_zero_shot(df, sam, out_dir="sample_3")

In [ ]:
out_df = pd.read_csv("sampled_dataframe_with_zeroshot_masks.csv")

In [ ]:
def generate_plots(df, ind):    
    # loop over each row in the dataframe
    row = df.iloc[ind]
    # extract image and mask paths
    hr_image_path = row['HR_path']
    hr_mask_path = row['HR_mask_path']

    lr_image_path = row['LR_output_path']
    lr_mask_path = row['LR_mask_path']

    # create the image comparison for HR
    hr_comparison = leafmap.image_comparison(
        hr_image_path,
        hr_mask_path,
        label1="HR Satellite Image",
        label2="HR Image Segmentation",
        show_labels=True
    )

    # create the image comparison for LR
    # lr_comparison = leafmap.image_comparison(
    #     lr_image_path,
    #     lr_mask_path,
    #     label1="LR Satellite Image",
    #     label2="LR Image Segmentation",
    #     show_labels=True,
    #      width= 100
    # )__
    print("IPCC Class: ", row['IPCC Class'])
    print("LCCS Class: ", row['LCCS class'])
    print("SMOD Class: ", row['SMOD Class'])
    print("Date: ", row['lowres_date'])
    print("lon, lat: ", row['lon'], row['lat'])
    return row

In [ ]:
result = generate_plots(out_df, 10)

In [ ]:
import rasterio
import numpy as np
from skimage import exposure
import skimage
png_path = result['HR_path']
with rasterio.open(png_path) as src:
    png_array = src.read()  # read all bands
    
epsg = 4326
origin = (-70.53959288131333, -13.022955494935278)  # roughly center of image
resolution = 0.0001	  # 1cm resolution 

height, width = png_array.shape[1:]
transform = rasterio.transform.from_origin(origin[0], origin[1], resolution, resolution)

meta = {
    'driver': 'GTiff',
    'dtype': png_array.dtype,  # use the dtype of your input array
    'crs': {'init': f'epsg:{epsg}'},
    'transform': transform,
    'height': height,
    'width': width,
    'count': png_array.shape[0]-1  # number of bands, the first dimension of the shape
}

with rasterio.open(result['HR_path'].split(".png")[0]+".tif", 'w', **meta) as dest:
    # Increase the brightness while clipping outliers
    clipped_image = exposure.equalize_hist(png_array[0:3,:,:])
    clipped_image = exposure.rescale_intensity(clipped_image, in_range='image', out_range=np.uint8)
    dest.write(clipped_image)
skimage.io.imsave(result['HR_path'].split(".png")[0]+"rescaled.png", clipped_image.transpose((1,2,0)))

In [ ]:
import skimage
skimage.io.imshow(clipped_image[0:3,:,:].transpose((1,2,0)))

In [ ]:
import leafmap.leafmap as leafmap
m = leafmap.Map(center=[origin[1], origin[0]], zoom=11, height="800px")
m.add_basemap("SATELLITE")
m

In [ ]:
import os
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = 'proxy/{port}'
m.layers[-1].visible = False
m.add_raster(result['HR_path'].split(".png")[0]+".tif", center=[origin[1], origin[0]], layer_name="Image")
m

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    sam_kwargs=None,
    automatic=False
)
sam.set_image(result['HR_path'].split(".png")[0]+".tif")
m = sam.show_map()
m

In [ ]:
m